In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from extension import proxies
from time import time, ctime, sleep
import logging
import threading


USERNAME = "sp47257626"
PASSWORD = "spsrawroao"
ENDPOINT = "au.smartproxy.com"
PORT_FIRST = 300183    # first port
PORT_INTERVAL = 2    # number of ports used in the same time in threading, recommend "2"
PORT_NUM = 10    # total number of ports, should be divisible by PORT_INTERVAL (PORT_NUM/PORT_INTERVAL=0)
URL = "https://www.bunnings.com.au/"    # "https://www.dulux.com.au//specifier/products/730" "https://www.bunnings.com.au/"
PRODUCTS = [
            ["Rust paint","Dulux 500ml Design Rust Effect Solution", 1],
            ["Wall Paint","British Paints Clean And Protect Low Sheen Extra Deep Interior Paint - 4L", 1],
            ["Wall Paint","Dulux 1L Interior Paint Wash&Wear Low Sheen Vivid White", 1],
            ["Wall Paint","Porter's Paints Eggshell Finish Aniseed 4L", 1]
           ]    # product list format: [search keyword, product name, page number]
PURCHASE_METHODS = ["Click & Collect", "Delivery"]    # bunnings purchase method list
INFO_TABS = ["Details", "Returns", "Ratings & Reviews"]    # bunnings info tab list
BREADCRUMBS = ["Interior Paint", "Paint", "Products"]    # bunnings purchase breadcrumb list


def bunnings_product_interact(username, password, endpoint, port, url, products, purchase_methods, info_tabs, breadcrumbs):
    
    for product in products:
        keyword = product[0]
        product_name = product[1]
        page = product[2]
        
        time_start = time()
        print("Port %d with product \"%s\" task starts at %s"%(port, product_name, ctime()))

        try:
            # open browser
            driver = open_browser(username, password, endpoint, port, url)
        
            # input keyword in search engine
            bunnings_search_engine_input(port, driver, keyword)
        
            # search
            search_engine_button_xpath = "//div[@class=\"HeaderSearchstyle__CancelSearchIconContainer-sc-3ryuy3-1 jyFvoF is-desktop\"]/button[@id=\"crossIcon\"]"
            inner_click(port, driver, search_engine_button_xpath, "searches", 15)
        
            # scholl to bottom
            scroll_to_bottom(port, driver)
            scroll_up(port, driver)
        
            # go to page
            if page != 1:
                if page < 6:
                    page_xpath = "//ul[@class=\"MuiPagination-ul\"]/li/button[@type=\"button\"][@aria-label=\"Go to page %d\"]"%page
                    inner_click(port, driver, page_xpath, "goes to page %d"%page, 15)
                    scroll_to_bottom(port, driver)
                    scroll_up(port, driver)
                else:
                    for j in range(5,page+1):
                        page_xpath = "//ul[@class=\"MuiPagination-ul\"]/li/button[@type=\"button\"][@aria-label=\"Go to page %d\"]"%j
                        inner_click(port, driver, page_xpath, "goes to page %d"%j, 15)
                        scroll_to_bottom(port, driver)
                        scroll_up(port, driver)
        
            # find product
            product_xpath = "//div[@class=\"product-wrapper\"]/div[@class=\"product-image-container\"]/a[@title=\"%s\"]"%product_name
            inner_click(port, driver, product_xpath, "finds product: %s"%product_name, 15)
            scroll_down(port, driver)
        
            # find purchase method
            for purchase_method in purchase_methods:
                purchase_method_xpath = "//div[@class=\"MuiTabs-flexContainer\"]/button/span[text()=\"%s\"]"%purchase_method
                inner_click(port, driver, purchase_method_xpath, "clicks \"%s\" tab"%purchase_method, 2)
                scroll_down(port, driver)        
            
            # scholl to bottom
            scroll_to_bottom(port, driver)
            scroll_up(port, driver)

            # find information
            for info_tab in info_tabs:
                info_tab_xpath = "//div[@aria-label=\"product-links\"][@class=\"MuiTabs-flexContainer\"]/button/span[text()=\"%s\"]"%info_tab
                inner_click(port, driver, info_tab_xpath, "clicks \"%s\" tab"%info_tab, 2)
                scroll_down(port, driver)
            
            # scholl to bottom
            scroll_to_bottom(port, driver)
            scroll_up(port, driver)

            # find breadcrumb
            for breadcrumb in breadcrumbs:
                breadcrumb_xpath = "//div[@class=\"MuiGrid-root MuiGrid-item\"]/nav[@aria-label=\"Breadcrumb\"]/ul/li/a/span[text()=\"%s\"]"%breadcrumb
                inner_click(port, driver, breadcrumb_xpath, "goes to \"%s\" page"%breadcrumb, 15)
                scroll_down(port, driver)
                go_back(port, driver)
                scroll_down(port, driver)
            
            # scholl to bottom
            scroll_to_bottom(port, driver)
            scroll_up(port, driver)
        
            # go to Home
            home_xpath = "//div[@class=\"MuiGrid-root MuiGrid-item\"]/nav[@aria-label=\"Breadcrumb\"]/ul/li/a[@data-locator=\"link_home\"][@aria-label=\"Home\"]"
            inner_click(port, driver, home_xpath, "goes to Home page", 15)
            scroll_down(port, driver)
        
            print("Port %d with product \"%s\" task Completed at %s!"%(port, product_name, ctime()))
    
        except:
            logging.exception('')
            print("Port %d with product \"%s\" task failed at %s!"%(port, product_name, ctime()))
    
        finally:
            close_browser(port, driver)
        
        time_end = time()
        print("Port %d with product \"%s\" task ends at %s"%(port, product_name, ctime()))
        print("Port %d with product \"%s\" task costs %d s"%(port, product_name, time_end-time_start))


def dulux_product_interact(username, password, endpoint, port, url):
    
    time_start = time()
    print("Port %d task starts at %s"%(port, ctime()))
    
    try:
        # open browser
        driver = open_browser(username, password, endpoint, port, url)

        # scholl to bottom
        scroll_to_bottom(port, driver)

        # find icon
        icons = ["icon icon-cta-map-pin","icon icon-cta-data-sheet","icon icon-cta-open-book", "icon icon-cta-desktop-download"]
        for icon in icons:
            icon_xpath = "//div[@class=\"quicklink__icon\"]/span/i[@class=\"%s\"]"%icon
            inner_click(port, driver, icon_xpath, "clicks \"%s\"!"%icon, 15)
            scroll_down(port, driver)
            go_back(port, driver)
            scroll_down(port, driver)
        
        # print('Task Completed at %s!'%ctime())
    
    except:
        print("Port %d task failed at %s!"%(port, ctime()))
    
    finally:
        close_browser(port, driver)

    time_end = time()
    print("Port %d task ends at %s"%(port, ctime()))
    print("Port %d task costs %d s"%(port, time_end-time_start))


def open_browser(username, password, endpoint, port, url):
    
    chrome_options = webdriver.ChromeOptions()    # create browser
    proxies_extension = proxies(username, password, endpoint, port)    # set proxy
    chrome_options.add_extension(proxies_extension)
    prefs = {"profile.managed_default_content_settings.images": 2, "permissions.default.stylesheet": 2}    # disable image & css loading
    chrome_options.add_experimental_option("prefs", prefs)
    # chrome_options.set_headless()    # hind window
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)    # create browser object
    driver.get(url)    # open website
    driver.maximize_window()    # maximize window
    sleep(2)
    print("Port %d opens browser successfully!"%port)
    return driver
    

def bunnings_search_engine_input(port, driver, keyword):
    
    # input keyword in the search engine
    search_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//input[@id=\"custom-css-outlined-input\"]")))
    search_input.send_keys(keyword)
    # driver.execute_script("arguments[0].setAttribute('value', '%s')"%keyword, search_input)
    sleep(5)
    print("Port %d input keyword in the search engine successfully!"%port)


def inner_click(port, driver, xpath, link_name, wait_time):
    
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "%s"%xpath)))
    driver.execute_script("arguments[0].scrollIntoView();", element)
    sleep(2)
    driver.execute_script("arguments[0].click();", element)
    sleep(wait_time)
    print("Port %d %s successfully!"%(port, link_name))


def scroll_up(port, driver):
    
    driver.execute_script("window.scrollBy(0,-500)")
    sleep(2)
    # print('Port %d scrolls up successfully!'%port)

    
def scroll_down(port, driver):
    
    driver.execute_script("window.scrollBy(0,500)")
    sleep(2)
    # print('Port %d scrolls down successfully!'%port)


def scroll_to_bottom(port, driver):
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)
    # print("Port %d scrolls down to the bottom successfully!"%port)


def go_back(port, driver):
    
    driver.back()
    sleep(15)
    print("Port %d goes back to the product page successfully!"%port)
    
    
def close_browser(port, driver):
    
    driver.delete_all_cookies()
    driver.quit()
    print("Port %d exits successfully!"%port)


def main(): 
    
    for i in range(PORT_NUM//PORT_INTERVAL):
    
        threads = []
    
        for j in range(PORT_INTERVAL):
            thread = threading.Thread(target = bunnings_product_interact, args = (USERNAME, PASSWORD, ENDPOINT, PORT_FIRST + PORT_INTERVAL*i + j, URL, PRODUCTS, PURCHASE_METHODS, INFO_TABS, BREADCRUMBS,))
            thread.start()
            threads.append(thread)
            # dulux_product_interact(USERNAME, PASSWORD, ENDPOINT, str(PORT_FIRST + j), URL)
            # bunnings_product_interact(USERNAME, PASSWORD, ENDPOINT, str(PORT_FIRST + j), URL, PRODUCTS, PURCHASE_METHODS, INFO_TABS, BREADCRUMBS)
    
        for thread in threads:
            thread.join()
        
    
if __name__ == "__main__":
    main()







====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472


Port 300183 with product "Dulux 500ml Design Rust Effect Solution" task starts at Mon Oct 18 14:13:48 2021Port 300184 with product "Dulux 500ml Design Rust Effect Solution" task starts at Mon Oct 18 14:13:48 2021



Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_mac64.zip
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_mac64.zip
Driver has been saved in cache [/Users/crystal/.wdm/drivers/chromedriver/mac64/91.0.4472.101]
Driver has been saved in cache [/Users/crystal/.wdm/drivers/chromedriver/mac64/91.0.4472.101]


Port 300184 opens browser successfully!
Port 300183 opens browser successfully!


ERROR:root:
Traceback (most recent call last):
  File "<ipython-input-1-48c71a8689d3>", line 45, in bunnings_product_interact
    bunnings_search_engine_input(port, driver, keyword)
  File "<ipython-input-1-48c71a8689d3>", line 178, in bunnings_search_engine_input
    search_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//input[@id=\"custom-css-outlined-input\"]")))
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/support/wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 

ERROR:root:
Traceback (most recent call last):
  File "<ipython-input-1-48c71a8689d3>", line 45, in bunnings_product_interact
    bunnings_search_engine_input(port, driver, keyword)
  File "<ipython-input-1-48c71a8689d3>", line 178, in bunnings_search_engine_input
    search_input = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//input[@id

...

Port 300183 with product "British Paints Clean And Protect Low Sheen Extra Deep Interior Paint - 4L" task failed at Mon Oct 18 14:16:09 2021!


ERROR:root:
Traceback (most recent call last):
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/Users/crystal/opt/anacond

Port 300184 with product "British Paints Clean And Protect Low Sheen Extra Deep Interior Paint - 4L" task failed at Mon Oct 18 14:16:09 2021!


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/Users/crystal/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/Users/c

KeyboardInterrupt: 